In [1]:
import pandas as pd
import numpy as np 
# import plotly
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
from nltk.probability import FreqDist
from os import path 
from PIL import Image 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib inline


# Data Cleaning

In [2]:
house_permit_from_file = pd.read_csv('Housing_Permits.csv',dtype = 'unicode')

In [3]:
house_permit_from_file = house_permit_from_file.filter(items = ['PropertyAddress','PermitDescription','Neighborhood'])

In [4]:
# Question 1 What types of construction permits are people pulling?
house_permit_from_file['PermitDescription'] = house_permit_from_file['PermitDescription'].astype(str)
house_permit_from_file = house_permit_from_file[house_permit_from_file['PermitDescription'].notnull()]

In [5]:
house_permit_from_file['lowercase'] = house_permit_from_file['PermitDescription'].str.lower()

In [6]:
house_permit_from_file['tokenized_text'] = house_permit_from_file['lowercase'].apply(word_tokenize)

In [7]:
house_permit_from_file_filter = house_permit_from_file.filter(items = ['PropertyAddress','lowercase','tokenized_text','Neighborhood']).reset_index()

In [8]:
stop_words=set(stopwords.words("english"))
house_permit_from_file_filter['no_stopwords'] = house_permit_from_file_filter['tokenized_text'].apply(lambda tokenized_list:[word for word in tokenized_list if word not in stop_words])

In [9]:
symbol_list = ["!", ")","(", ":", ",",".", "...","-", "|", "'",'"','’','…',"“","”","&","^",";","@","#","$","%","_","+","="]
stop_words.update(symbol_list)
house_permit_from_file_filter['no_symbols'] = house_permit_from_file_filter['no_stopwords'].apply(lambda x: [word for word in x if word not in stop_words])
house_permit_from_file_filter['no_symbols'] = house_permit_from_file_filter['no_symbols'].apply(lambda x: [word for word in x if not word.isnumeric()])

In [10]:
common_word_list = ['per','code','install','new','permit','work']
common_word_set = set(common_word_list)
house_permit_from_file_filter['remove_common_word'] = house_permit_from_file_filter['no_symbols'].apply(lambda x: [word for word in x if word not in common_word_set])

In [11]:
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['no_stopwords'].apply(lambda x: [word for word in x if word not in stop_words])
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['tokenized_final'].apply(lambda x: [word for word in x if not word.isnumeric()])
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['tokenized_final'].apply(lambda x: [word for word in x if word not in common_word_set])

In [12]:
house_permit_analysis = house_permit_from_file_filter.filter(items = ['PropertyAddress','tokenized_final','Neighborhood'])

In [13]:
house_permit_analysis = house_permit_analysis.rename(columns = {'PropertyAddress':'address'})

In [14]:
house_permit_analysis.to_csv('house_permit_analysis.csv')

In [15]:
house_permit_analysis.head()

,address,tokenized_final,Neighborhood
0,1752 N GAY ST,"[interior, exterior, alteration, existing, bui...",BROADWAY EAST
1,3401 LABYRINTH RD,"[demolition, building/structure, wood, length:...",CROSS COUNTRY
2,815 BROOKS LANE,"[windows, 2-new, exterior, doors, ceramic, flo...",RESERVOIR HILL
3,1743 DRUID HILL AVE,"[*authorization, proceed*, sanitary, line, rea...",UPTON
4,6026 PINEHURST RD,"[plumbing, fixtures, sanitary, connection, rec...",LAKE EVESHAM


# Neighborhood Keyword Freq Analysis

In [26]:
ABELL = ['ABELL']
ALLENDALE = ['ALLENDALE']
ARCADIA = ['ARCADIA']
ARLINGTON = ['ARLINGTON']
ARMISTEADGARDENS = ['ARMISTEAD GARDENS']
ASHBURTON = ['ASHBURTON']
BALTIMOREHIGHLANDS = ['BALTIMORE HIGHLANDS']
BALTIMORELINWOOD = ['BALTIMORE-LINWOOD']
BARCLAY = ['BARCLAY']
BARRECIRCLE = ['BARRE CIRCLE']
BEECHFIELD = ['BEECHFIELD']
BELAIREDISON = ['BELAIR-EDISON']
BELAIRPARKSIDE = ['BELAIR-PARKSIDE']
BELLONAGITTINGS = ['BELLONA-GITTINGS']
BEREA = ['BEREA']
BETTERWAVERLY = ['BETTER WAVERLY']
BEVERLYHILLS = ['BEVERLY HILLS']
BIDDLESTREET = ['BIDDLE STREET']
BLYTHEWOOD = ['BLYTHEWOOD']
BOLTONHILL = ['BOLTON HILL']
BOOTHBOYD = ['BOOTH-BOYD']
BREWERSHILL = ['BREWERS HILL']
BRIDGEVIEWGREENLAWN = ['BRIDGEVIEW/GREENLAWN']
BROADWAYEAST = ['BROADWAY EAST']
BROENINGMANOR = ['BROENING MANOR']
BROOKLYN = ['BROOKLYN']
BURLEITHLEIGHTON = ['BURLEITH-LEIGHTON']
#BUTCHERSHILL = ['BUTCHER'S HILL']
CALLAWAYGARRISON = ['CALLAWAY-GARRISON']
CAMERONVILLAGE = ['CAMERON VILLAGE']
CANTON = ['CANTON']
CANTONINDUSTRIALAREA = ['CANTON INDUSTRIAL AREA']
CARROLLCAMDENINDUSTRIALAR = ['CARROLL - CAMDEN INDUSTRIAL AR']
CARROLLPARK = ['CARROLL PARK']
CARROLLSOUTHHILTON = ['CARROLL-SOUTH HILTON']
CARROLLTONRIDGE = ['CARROLLTON RIDGE']
CEDARCROFT = ['CEDARCROFT']
CEDMONT = ['CEDMONT']
CEDONIA = ['CEDONIA']
CENTRALFORESTPARK = ['CENTRAL FOREST PARK']
CENTRALPARKHEIGHTS = ['CENTRAL PARK HEIGHTS']
CHARLESNORTH = ['CHARLES NORTH']
CHARLESVILLAGE = ['CHARLES VILLAGE']
CHERRYHILL = ['CHERRY HILL']
CHESWOLDE = ['CHESWOLDE']
CHINQUAPINPARKBELVEDERE = ['CHINQUAPIN PARK-BELVEDERE']
CHRISTOPHER = ['CHRISTOPHER']
CLAREMONT = ['CLAREMONT']
CLAREMONTFREEDOM = ['CLAREMONT-FREEDOM']
CLIFTONPARK = ['CLIFTON PARK']
COLDSPRING = ['COLDSPRING']
COLDSTREAMHOMESTEADMONTEBELL = ['COLDSTREAM HOMESTEAD MONTEBELL']
CONCERNEDCITIZENSOFFORESTP = ['CONCERNED CITIZENS OF FOREST P']
COPPINHEIGHTSASHCOEAST = ['COPPIN HEIGHTS/ASH-CO-EAST']
CROSSCOUNTRY = ['CROSS COUNTRY']
CROSSKEYS = ['CROSS KEYS']
CURTISBAY = ['CURTIS BAY']
CURTISBAYINDUSTRIAL = ['CURTIS BAY INDUSTRIAL']
CYLBURN = ['CYLBURN']
DARLEYPARK = ['DARLEY PARK']
DICKEYVILLE = ['DICKEYVILLE']
DOLFIELD = ['DOLFIELD']
DORCHESTER = ['DORCHESTER']
DOWNTOWN = ['DOWNTOWN']
DRUIDHEIGHTS = ['DRUID HEIGHTS']
DRUIDHILLPARK = ['DRUID HILL PARK']
DUNBARBROADWAY = ['DUNBAR-BROADWAY']
DUNDALKMARINETERMINAL = ['DUNDALK MARINE TERMINAL']
EASTARLINGTON = ['EAST ARLINGTON']
EASTBALTIMOREMIDWAY = ['EAST BALTIMORE MIDWAY']
EASTERWOOD = ['EASTERWOOD']
EASTWOOD = ['EASTWOOD']
EDMONDSONVILLAGE = ['EDMONDSON VILLAGE']
EDNORGARDENSLAKESIDE = ['EDNOR GARDENS-LAKESIDE']
ELLWOODPARKMONUMENT = ['ELLWOOD PARK/MONUMENT']
EVERGREEN = ['EVERGREEN']
EVERGREENLAWN = ['EVERGREEN LAWN']
FAIRFIELDAREA = ['FAIRFIELD AREA']
FAIRMONT = ['FAIRMONT']
FALLSTAFF = ['FALLSTAFF']
FEDERALHILL = ['FEDERAL HILL']
FEDERALHILLMONTGOMERY = ['FEDERAL HILL/MONTGOMERY']
FELLSPOINT = ['FELLS POINT']
FORESTPARK = ['FOREST PARK']
FORESTPARKGOLFCOURSE = ['FOREST PARK GOLF COURSE']
FOURBYFOUR = ['FOUR BY FOUR']
FRANKFORD = ['FRANKFORD']
FRANKLINSQUARE = ['FRANKLIN SQUARE']
FRANKLINTOWN = ['FRANKLINTOWN']
FRANKLINTOWNROAD = ['FRANKLINTOWN ROAD']
GARWYNOAKS = ['GARWYN OAKS']
GAYSTREET = ['GAY STREET']
GLEN = ['GLEN']
GLENOAKS = ['GLEN OAKS']
GLENHAMBELFORD = ['GLENHAM-BELFORD']
GRACELANDPARK = ['GRACELAND PARK']
GREEKTOWN = ['GREEKTOWN']
GREENMOUNTCEMETERY = ['GREENMOUNT CEMETERY']
GREENMOUNTWEST = ['GREENMOUNT WEST']
GREENSPRING = ['GREENSPRING']
GROVEPARK = ['GROVE PARK']
GUILFORD = ['GUILFORD']
GWYNNSFALLS = ['GWYNNS FALLS']
GWYNNSFALLSLEAKINPARK = ['GWYNNS FALLS/LEAKIN PARK']
HAMPDEN = ['HAMPDEN']
HANLONLONGWOOD = ['HANLON-LONGWOOD']
HARFORDECHODALEPERRINGPARKW = ['HARFORD-ECHODALE/PERRING PARKW']
HARLEMPARK = ['HARLEM PARK']
HARWOOD = ['HARWOOD']
HAWKINSPOINT = ['HAWKINS POINT']
HERITAGECROSSING = ['HERITAGE CROSSING']
HIGHLANDTOWN = ['HIGHLANDTOWN']
HILLEN = ['HILLEN']
HOESHEIGHTS = ['HOES HEIGHTS']
HOLABIRDINDUSTRIALPARK = ['HOLABIRD INDUSTRIAL PARK']
HOLLINSMARKET = ['HOLLINS MARKET']
HOMELAND = ['HOMELAND']
HOPKINSBAYVIEW = ['HOPKINS BAYVIEW']
HOWARDPARK = ['HOWARD PARK']
HUNTINGRIDGE = ['HUNTING RIDGE']
IDLEWOOD = ['IDLEWOOD']
INNERHARBOR = ['INNER HARBOR']
IRVINGTON = ['IRVINGTON']
JOHNSHOPKINSHOMEWOOD = ['JOHNS HOPKINS HOMEWOOD']
JOHNSTONSQUARE = ['JOHNSTON SQUARE']
JONESFALLSAREA = ['JONES FALLS AREA']
JONESTOWN = ['JONESTOWN']
JOSEPHLEE = ['JOSEPH LEE']
KENILWORTHPARK = ['KENILWORTH PARK']
KERNEWOOD = ['KERNEWOOD']
KESWICK = ['KESWICK']
KRESSON = ['KRESSON']
LAFAYETTE = ['LAFAYETTE']
LAKEEVESHAM = ['LAKE EVESHAM']
LAKEWALKER = ['LAKE WALKER']
LAKELAND = ['LAKELAND']
LANGSTONHUGHES = ['LANGSTON HUGHES']
LAURAVILLE = ['LAURAVILLE']
LEVINDALE = ['LEVINDALE']
LIBERTYSQUARE = ['LIBERTY SQUARE']
LITTLEITALY = ['LITTLE ITALY']
LOCHRAVEN = ['LOCH RAVEN']
LOCUSTPOINT = ['LOCUST POINT']
LOCUSTPOINTINDUSTRIALAREA = ['LOCUST POINT INDUSTRIAL AREA']
LOWERHERRINGRUNPARK = ['LOWER HERRING RUN PARK']
LOYOLANOTREDAME = ['LOYOLA/NOTRE DAME']
LUCILLEPARK = ['LUCILLE PARK']
MADISONPARK = ['MADISON PARK']
MADISONEASTEND = ['MADISON-EASTEND']
MAYFIELD = ['MAYFIELD']
MCELDERRYPARK = ['MCELDERRY PARK']
MEDFIELD = ['MEDFIELD']
MEDFORD = ['MEDFORD']
MIDDLEBRANCHREEDBIRDPARKS = ['MIDDLE BRANCH/REEDBIRD PARKS']
MIDDLEEAST = ['MIDDLE EAST']
MIDGOVANS = ['MID-GOVANS']
MIDTOWNBELVEDERE = ['MID-TOWN BELVEDERE']
MIDTOWNEDMONDSON = ['MIDTOWN-EDMONDSON']
MILLHILL = ['MILLHILL']
MILTONMONTFORD = ['MILTON-MONTFORD']
MONDAWMIN = ['MONDAWMIN']
MONTEBELLO = ['MONTEBELLO']
MORAVIAWALTHER = ['MORAVIA-WALTHER']
MORGANPARK = ['MORGAN PARK']
MORGANSTATEUNIVERSITY = ['MORGAN STATE UNIVERSITY']
MORRELLPARK = ['MORRELL PARK']
MOSHER = ['MOSHER']
MOUNTHOLLY = ['MOUNT HOLLY']
MOUNTVERNON = ['MOUNT VERNON']
MOUNTWASHINGTON = ['MOUNT WASHINGTON']
MOUNTWINANS = ['MOUNT WINANS']
MTPLEASANTPARK = ['MT PLEASANT PARK']
NEWNORTHWOOD = ['NEW NORTHWOOD']
NEWSOUTHWESTMOUNTCLARE = ['NEW SOUTHWEST/MOUNT CLARE']
NORTHHARFORDROAD = ['NORTH HARFORD ROAD']
NORTHROLANDPARKPOPLARHILL = ['NORTH ROLAND PARK/POPLAR HILL']
NORTHWESTCOMMUNITYACTION = ['NORTHWEST COMMUNITY ACTION']
OAKENSHAWE = ['OAKENSHAWE']
OAKLEE = ['OAKLEE']
#ODONNELLHEIGHTS = ['O'DONNELL HEIGHTS']
OLDTOWN = ['OLDTOWN']
OLIVER = ['OLIVER']
ORANGEVILLE = ['ORANGEVILLE']
ORANGEVILLEINDUSTRIALAREA = ['ORANGEVILLE INDUSTRIAL AREA']
ORIGINALNORTHWOOD = ['ORIGINAL NORTHWOOD']
OTTERBEIN = ['OTTERBEIN']
OVERLEA = ['OVERLEA']
PANWAYBRADDISHAVENUE = ['PANWAY/BRADDISH AVENUE']
PARKCIRCLE = ['PARK CIRCLE']
PARKLANE = ['PARKLANE']
PARKSIDE = ['PARKSIDE']
PARKVIEWWOODBROOK = ['PARKVIEW/WOODBROOK']
PATTERSONPARK = ['PATTERSON PARK']
PATTERSONPLACE = ['PATTERSON PLACE']
PENLUCY = ['PEN LUCY']
PENNNORTH = ['PENN NORTH']
PENNFALLSWAY = ['PENN-FALLSWAY']
PENROSE = ['PENROSE']
PENROSEFAYETTESTREETOUTREAC = ['PENROSE/FAYETTE STREET OUTREAC']
PERKINSHOMES = ['PERKINS HOMES']
PERRINGLOCH = ['PERRING LOCH']
PIMLICOGOODNEIGHBORS = ['PIMLICO GOOD NEIGHBORS']
PLEASANTVIEWGARDENS = ['PLEASANT VIEW GARDENS']
POPPLETON = ['POPPLETON']
PORTCOVINGTON = ['PORT COVINGTON']
PULASKIINDUSTRIALAREA = ['PULASKI INDUSTRIAL AREA']
PURNELL = ['PURNELL']
RADNORWINSTON = ['RADNOR-WINSTON']
RAMBLEWOOD = ['RAMBLEWOOD']
REISTERSTOWNSTATION = ['REISTERSTOWN STATION']
REMINGTON = ['REMINGTON']
RESERVOIRHILL = ['RESERVOIR HILL']
RICHNORSPRINGS = ['RICHNOR SPRINGS']
#RIDGELYSDELIGHT = ['RIDGELY'S DELIGHT']
RIVERSIDE = ['RIVERSIDE']
ROGNELHEIGHTS = ['ROGNEL HEIGHTS']
ROLANDPARK = ['ROLAND PARK']
ROLANDPARKPOPLARHILL = ['ROLAND PARK/POPLAR HILL']
ROSEBANK = ['ROSEBANK']
ROSEMONT = ['ROSEMONT']
ROSEMONTEAST = ['ROSEMONT EAST']
ROSEMONTHOMEOWNERSTENANTS = ['ROSEMONT HOMEOWNERS/TENANTS']
SABINAMATTFELDT = ['SABINA-MATTFELDT']
SAINTAGNES = ['SAINT AGNES']
SAINTHELENA = ['SAINT HELENA']
SAINTJOSEPHS = ['SAINT JOSEPHS']
SAINTPAUL = ['SAINT PAUL']
SANDTOWNWINCHESTER = ['SANDTOWN-WINCHESTER']
SBIC = ['SBIC']
SETONBUSINESSPARK = ['SETON BUSINESS PARK']
SETONHILL = ['SETON HILL']
SHARPLEADENHALL = ['SHARP-LEADENHALL']
SHIPLEYHILL = ['SHIPLEY HILL']
SOUTHCLIFTONPARK = ['SOUTH CLIFTON PARK']
SPRINGGARDENINDUSTRIALAREA = ['SPRING GARDEN INDUSTRIAL AREA']
STADIUMAREA = ['STADIUM AREA']
STONEWOODPENTWOODWINSTON = ['STONEWOOD-PENTWOOD-WINSTON']
TAYLORHEIGHTS = ['TAYLOR HEIGHTS']
TENHILLS = ['TEN HILLS']
THEORCHARDS = ['THE ORCHARDS']
TOWANDAGRANTLEY = ['TOWANDA-GRANTLEY']
TREMONT = ['TREMONT']
TUSCANYCANTERBURY = ['TUSCANY-CANTERBURY']
UNIONSQUARE = ['UNION SQUARE']
UNIVERSITYOFMARYLAND = ['UNIVERSITY OF MARYLAND']
UPLANDS = ['UPLANDS']
UPPERFELLSPOINT = ['UPPER FELLS POINT']
UPTON = ['UPTON']
VILLAGESOFHOMELAND = ['VILLAGES OF HOMELAND']
VIOLETVILLE = ['VIOLETVILLE']
WAKEFIELD = ['WAKEFIELD']
WALBROOK = ['WALBROOK']
WALTHERSON = ['WALTHERSON']
WASHINGTONHILL = ['WASHINGTON HILL']
WASHINGTONVILLAGE = ['WASHINGTON VILLAGE']
WAVERLY = ['WAVERLY']
WESTARLINGTON = ['WEST ARLINGTON']
WESTFORESTPARK = ['WEST FOREST PARK']
WESTHILLS = ['WEST HILLS']
WESTFIELD = ['WESTFIELD']
WESTGATE = ['WESTGATE']
WESTPORT = ['WESTPORT']
WILSONHEIGHTS = ['WILSON HEIGHTS']
WILSONPARK = ['WILSON PARK']
WINCHESTER = ['WINCHESTER']
WINDSORHILLS = ['WINDSOR HILLS']
WINSTONGOVANS = ['WINSTON-GOVANS']
WOODBERRY = ['WOODBERRY']
WOODBOURNEHEIGHTS = ['WOODBOURNE HEIGHTS']
WOODBOURNEMCCABE = ['WOODBOURNE-MCCABE']
WOODMERE = ['WOODMERE']
WOODRING = ['WOODRING']
WYMANPARK = ['WYMAN PARK']
WYNDHURST = ['WYNDHURST']
YALEHEIGHTS = ['YALE HEIGHTS']

# selecting rows based on condition 
ABELL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ABELL)]
ALLENDALE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ALLENDALE)]
ARCADIA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ARCADIA)]
ARLINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ARLINGTON)]
ARMISTEADGARDENS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ARMISTEADGARDENS)]
ASHBURTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ASHBURTON)]
BALTIMOREHIGHLANDS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BALTIMOREHIGHLANDS)]
BALTIMORELINWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BALTIMORELINWOOD)]
BARCLAY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BARCLAY)]
BARRECIRCLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BARRECIRCLE)]
BEECHFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BEECHFIELD)]
BELAIREDISON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BELAIREDISON)]
BELAIRPARKSIDE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BELAIRPARKSIDE)]
BELLONAGITTINGS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BELLONAGITTINGS)]
BEREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BEREA)]
BETTERWAVERLY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BETTERWAVERLY)]
BEVERLYHILLS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BEVERLYHILLS)]
BIDDLESTREET_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BIDDLESTREET)]
BLYTHEWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BLYTHEWOOD)]
BOLTONHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BOLTONHILL)]
BOOTHBOYD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BOOTHBOYD)]
BREWERSHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BREWERSHILL)]
BRIDGEVIEWGREENLAWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BRIDGEVIEWGREENLAWN)]
BROADWAYEAST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BROADWAYEAST)]
BROENINGMANOR_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BROENINGMANOR)]
BROOKLYN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BROOKLYN)]
BURLEITHLEIGHTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BURLEITHLEIGHTON)]
#BUTCHERSHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(BUTCHERSHILL)]
CALLAWAYGARRISON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CALLAWAYGARRISON)]
CAMERONVILLAGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CAMERONVILLAGE)]
CANTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CANTON)]
CANTONINDUSTRIALAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CANTONINDUSTRIALAREA)]
CARROLLCAMDENINDUSTRIALAR_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CARROLLCAMDENINDUSTRIALAR)]
CARROLLPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CARROLLPARK)]
CARROLLSOUTHHILTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CARROLLSOUTHHILTON)]
CARROLLTONRIDGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CARROLLTONRIDGE)]
CEDARCROFT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CEDARCROFT)]
CEDMONT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CEDMONT)]
CEDONIA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CEDONIA)]
CENTRALFORESTPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CENTRALFORESTPARK)]
CENTRALPARKHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CENTRALPARKHEIGHTS)]
CHARLESNORTH_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHARLESNORTH)]
CHARLESVILLAGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHARLESVILLAGE)]
CHERRYHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHERRYHILL)]
CHESWOLDE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHESWOLDE)]
CHINQUAPINPARKBELVEDERE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHINQUAPINPARKBELVEDERE)]
CHRISTOPHER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CHRISTOPHER)]
CLAREMONT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CLAREMONT)]
CLAREMONTFREEDOM_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CLAREMONTFREEDOM)]
CLIFTONPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CLIFTONPARK)]
COLDSPRING_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(COLDSPRING)]
COLDSTREAMHOMESTEADMONTEBELL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(COLDSTREAMHOMESTEADMONTEBELL)]
CONCERNEDCITIZENSOFFORESTP_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CONCERNEDCITIZENSOFFORESTP)]
COPPINHEIGHTSASHCOEAST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(COPPINHEIGHTSASHCOEAST)]
CROSSCOUNTRY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CROSSCOUNTRY)]
CROSSKEYS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CROSSKEYS)]
CURTISBAY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CURTISBAY)]
CURTISBAYINDUSTRIAL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CURTISBAYINDUSTRIAL)]
CYLBURN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(CYLBURN)]
DARLEYPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DARLEYPARK)]
DICKEYVILLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DICKEYVILLE)]
DOLFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DOLFIELD)]
DORCHESTER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DORCHESTER)]
DOWNTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DOWNTOWN)]
DRUIDHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DRUIDHEIGHTS)]
DRUIDHILLPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DRUIDHILLPARK)]
DUNBARBROADWAY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DUNBARBROADWAY)]
DUNDALKMARINETERMINAL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(DUNDALKMARINETERMINAL)]
EASTARLINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EASTARLINGTON)]
EASTBALTIMOREMIDWAY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EASTBALTIMOREMIDWAY)]
EASTERWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EASTERWOOD)]
EASTWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EASTWOOD)]
EDMONDSONVILLAGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EDMONDSONVILLAGE)]
EDNORGARDENSLAKESIDE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EDNORGARDENSLAKESIDE)]
ELLWOODPARKMONUMENT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ELLWOODPARKMONUMENT)]
EVERGREEN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EVERGREEN)]
EVERGREENLAWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(EVERGREENLAWN)]
FAIRFIELDAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FAIRFIELDAREA)]
FAIRMONT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FAIRMONT)]
FALLSTAFF_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FALLSTAFF)]
FEDERALHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FEDERALHILL)]
FEDERALHILLMONTGOMERY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FEDERALHILLMONTGOMERY)]
FELLSPOINT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FELLSPOINT)]
FORESTPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FORESTPARK)]
FORESTPARKGOLFCOURSE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FORESTPARKGOLFCOURSE)]
FOURBYFOUR_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FOURBYFOUR)]
FRANKFORD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FRANKFORD)]
FRANKLINSQUARE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FRANKLINSQUARE)]
FRANKLINTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FRANKLINTOWN)]
FRANKLINTOWNROAD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(FRANKLINTOWNROAD)]
GARWYNOAKS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GARWYNOAKS)]
GAYSTREET_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GAYSTREET)]
GLEN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GLEN)]
GLENOAKS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GLENOAKS)]
GLENHAMBELFORD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GLENHAMBELFORD)]
GRACELANDPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GRACELANDPARK)]
GREEKTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GREEKTOWN)]
GREENMOUNTCEMETERY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GREENMOUNTCEMETERY)]
GREENMOUNTWEST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GREENMOUNTWEST)]
GREENSPRING_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GREENSPRING)]
GROVEPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GROVEPARK)]
GUILFORD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GUILFORD)]
GWYNNSFALLS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GWYNNSFALLS)]
GWYNNSFALLSLEAKINPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(GWYNNSFALLSLEAKINPARK)]
HAMPDEN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HAMPDEN)]
HANLONLONGWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HANLONLONGWOOD)]
HARFORDECHODALEPERRINGPARKW_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HARFORDECHODALEPERRINGPARKW)]
HARLEMPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HARLEMPARK)]
HARWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HARWOOD)]
HAWKINSPOINT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HAWKINSPOINT)]
HERITAGECROSSING_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HERITAGECROSSING)]
HIGHLANDTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HIGHLANDTOWN)]
HILLEN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HILLEN)]
HOESHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOESHEIGHTS)]
HOLABIRDINDUSTRIALPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOLABIRDINDUSTRIALPARK)]
HOLLINSMARKET_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOLLINSMARKET)]
HOMELAND_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOMELAND)]
HOPKINSBAYVIEW_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOPKINSBAYVIEW)]
HOWARDPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HOWARDPARK)]
HUNTINGRIDGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(HUNTINGRIDGE)]
IDLEWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(IDLEWOOD)]
INNERHARBOR_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(INNERHARBOR)]
IRVINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(IRVINGTON)]
JOHNSHOPKINSHOMEWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(JOHNSHOPKINSHOMEWOOD)]
JOHNSTONSQUARE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(JOHNSTONSQUARE)]
JONESFALLSAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(JONESFALLSAREA)]
JONESTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(JONESTOWN)]
JOSEPHLEE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(JOSEPHLEE)]
KENILWORTHPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(KENILWORTHPARK)]
KERNEWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(KERNEWOOD)]
KESWICK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(KESWICK)]
KRESSON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(KRESSON)]
LAFAYETTE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LAFAYETTE)]
LAKEEVESHAM_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LAKEEVESHAM)]
LAKEWALKER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LAKEWALKER)]
LAKELAND_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LAKELAND)]
LANGSTONHUGHES_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LANGSTONHUGHES)]
LAURAVILLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LAURAVILLE)]
LEVINDALE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LEVINDALE)]
LIBERTYSQUARE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LIBERTYSQUARE)]
LITTLEITALY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LITTLEITALY)]
LOCHRAVEN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LOCHRAVEN)]
LOCUSTPOINT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LOCUSTPOINT)]
LOCUSTPOINTINDUSTRIALAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LOCUSTPOINTINDUSTRIALAREA)]
LOWERHERRINGRUNPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LOWERHERRINGRUNPARK)]
LOYOLANOTREDAME_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LOYOLANOTREDAME)]
LUCILLEPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(LUCILLEPARK)]
MADISONPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MADISONPARK)]
MADISONEASTEND_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MADISONEASTEND)]
MAYFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MAYFIELD)]
MCELDERRYPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MCELDERRYPARK)]
MEDFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MEDFIELD)]
MEDFORD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MEDFORD)]
MIDDLEBRANCHREEDBIRDPARKS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MIDDLEBRANCHREEDBIRDPARKS)]
MIDDLEEAST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MIDDLEEAST)]
MIDGOVANS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MIDGOVANS)]
MIDTOWNBELVEDERE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MIDTOWNBELVEDERE)]
MIDTOWNEDMONDSON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MIDTOWNEDMONDSON)]
MILLHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MILLHILL)]
MILTONMONTFORD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MILTONMONTFORD)]
MONDAWMIN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MONDAWMIN)]
MONTEBELLO_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MONTEBELLO)]
MORAVIAWALTHER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MORAVIAWALTHER)]
MORGANPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MORGANPARK)]
MORGANSTATEUNIVERSITY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MORGANSTATEUNIVERSITY)]
MORRELLPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MORRELLPARK)]
MOSHER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MOSHER)]
MOUNTHOLLY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MOUNTHOLLY)]
MOUNTVERNON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MOUNTVERNON)]
MOUNTWASHINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MOUNTWASHINGTON)]
MOUNTWINANS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MOUNTWINANS)]
MTPLEASANTPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(MTPLEASANTPARK)]
NEWNORTHWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(NEWNORTHWOOD)]
NEWSOUTHWESTMOUNTCLARE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(NEWSOUTHWESTMOUNTCLARE)]
NORTHHARFORDROAD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(NORTHHARFORDROAD)]
NORTHROLANDPARKPOPLARHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(NORTHROLANDPARKPOPLARHILL)]
NORTHWESTCOMMUNITYACTION_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(NORTHWESTCOMMUNITYACTION)]
OAKENSHAWE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OAKENSHAWE)]
OAKLEE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OAKLEE)]
#ODONNELLHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ODONNELLHEIGHTS)]
OLDTOWN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OLDTOWN)]
OLIVER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OLIVER)]
ORANGEVILLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ORANGEVILLE)]
ORANGEVILLEINDUSTRIALAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ORANGEVILLEINDUSTRIALAREA)]
ORIGINALNORTHWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ORIGINALNORTHWOOD)]
OTTERBEIN_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OTTERBEIN)]
OVERLEA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(OVERLEA)]
PANWAYBRADDISHAVENUE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PANWAYBRADDISHAVENUE)]
PARKCIRCLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PARKCIRCLE)]
PARKLANE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PARKLANE)]
PARKSIDE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PARKSIDE)]
PARKVIEWWOODBROOK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PARKVIEWWOODBROOK)]
PATTERSONPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PATTERSONPARK)]
PATTERSONPLACE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PATTERSONPLACE)]
PENLUCY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PENLUCY)]
PENNNORTH_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PENNNORTH)]
PENNFALLSWAY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PENNFALLSWAY)]
PENROSE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PENROSE)]
PENROSEFAYETTESTREETOUTREAC_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PENROSEFAYETTESTREETOUTREAC)]
PERKINSHOMES_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PERKINSHOMES)]
PERRINGLOCH_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PERRINGLOCH)]
PIMLICOGOODNEIGHBORS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PIMLICOGOODNEIGHBORS)]
PLEASANTVIEWGARDENS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PLEASANTVIEWGARDENS)]
POPPLETON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(POPPLETON)]
PORTCOVINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PORTCOVINGTON)]
PULASKIINDUSTRIALAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PULASKIINDUSTRIALAREA)]
PURNELL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(PURNELL)]
RADNORWINSTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RADNORWINSTON)]
RAMBLEWOOD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RAMBLEWOOD)]
REISTERSTOWNSTATION_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(REISTERSTOWNSTATION)]
REMINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(REMINGTON)]
RESERVOIRHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RESERVOIRHILL)]
RICHNORSPRINGS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RICHNORSPRINGS)]
#RIDGELYSDELIGHT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RIDGELYSDELIGHT)]
RIVERSIDE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(RIVERSIDE)]
ROGNELHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROGNELHEIGHTS)]
ROLANDPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROLANDPARK)]
ROLANDPARKPOPLARHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROLANDPARKPOPLARHILL)]
ROSEBANK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROSEBANK)]
ROSEMONT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROSEMONT)]
ROSEMONTEAST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROSEMONTEAST)]
ROSEMONTHOMEOWNERSTENANTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(ROSEMONTHOMEOWNERSTENANTS)]
SABINAMATTFELDT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SABINAMATTFELDT)]
SAINTAGNES_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SAINTAGNES)]
SAINTHELENA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SAINTHELENA)]
SAINTJOSEPHS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SAINTJOSEPHS)]
SAINTPAUL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SAINTPAUL)]
SANDTOWNWINCHESTER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SANDTOWNWINCHESTER)]
SBIC_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SBIC)]
SETONBUSINESSPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SETONBUSINESSPARK)]
SETONHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SETONHILL)]
SHARPLEADENHALL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SHARPLEADENHALL)]
SHIPLEYHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SHIPLEYHILL)]
SOUTHCLIFTONPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SOUTHCLIFTONPARK)]
SPRINGGARDENINDUSTRIALAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(SPRINGGARDENINDUSTRIALAREA)]
STADIUMAREA_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(STADIUMAREA)]
STONEWOODPENTWOODWINSTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(STONEWOODPENTWOODWINSTON)]
TAYLORHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(TAYLORHEIGHTS)]
TENHILLS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(TENHILLS)]
THEORCHARDS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(THEORCHARDS)]
TOWANDAGRANTLEY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(TOWANDAGRANTLEY)]
TREMONT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(TREMONT)]
TUSCANYCANTERBURY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(TUSCANYCANTERBURY)]
UNIONSQUARE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(UNIONSQUARE)]
UNIVERSITYOFMARYLAND_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(UNIVERSITYOFMARYLAND)]
UPLANDS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(UPLANDS)]
UPPERFELLSPOINT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(UPPERFELLSPOINT)]
UPTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(UPTON)]
VILLAGESOFHOMELAND_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(VILLAGESOFHOMELAND)]
VIOLETVILLE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(VIOLETVILLE)]
WAKEFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WAKEFIELD)]
WALBROOK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WALBROOK)]
WALTHERSON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WALTHERSON)]
WASHINGTONHILL_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WASHINGTONHILL)]
WASHINGTONVILLAGE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WASHINGTONVILLAGE)]
WAVERLY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WAVERLY)]
WESTARLINGTON_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTARLINGTON)]
WESTFORESTPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTFORESTPARK)]
WESTHILLS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTHILLS)]
WESTFIELD_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTFIELD)]
WESTGATE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTGATE)]
WESTPORT_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WESTPORT)]
WILSONHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WILSONHEIGHTS)]
WILSONPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WILSONPARK)]
WINCHESTER_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WINCHESTER)]
WINDSORHILLS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WINDSORHILLS)]
WINSTONGOVANS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WINSTONGOVANS)]
WOODBERRY_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WOODBERRY)]
WOODBOURNEHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WOODBOURNEHEIGHTS)]
WOODBOURNEMCCABE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WOODBOURNEMCCABE)]
WOODMERE_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WOODMERE)]
WOODRING_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WOODRING)]
WYMANPARK_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WYMANPARK)]
WYNDHURST_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(WYNDHURST)]
YALEHEIGHTS_df = neighborhoods.loc[neighborhoods['Neighborhood'].isin(YALEHEIGHTS)]

In [31]:
description_list_final_1 = ABELL_df['tokenized_final'].tolist()
description_list_final_2 = ALLENDALE_df['tokenized_final'].tolist()
description_list_final_3 = ARCADIA_df['tokenized_final'].tolist()
description_list_final_4 = ARLINGTON_df['tokenized_final'].tolist()
description_list_final_5 = ARMISTEADGARDENS_df['tokenized_final'].tolist()
description_list_final_6 = ASHBURTON_df['tokenized_final'].tolist()
description_list_final_7 = BALTIMOREHIGHLANDS_df['tokenized_final'].tolist()
description_list_final_8 = BALTIMORELINWOOD_df['tokenized_final'].tolist()
description_list_final_9 = BARCLAY_df['tokenized_final'].tolist()
description_list_final_10 = BARRECIRCLE_df['tokenized_final'].tolist()
description_list_final_11 = BEECHFIELD_df['tokenized_final'].tolist()
description_list_final_12 = BELAIREDISON_df['tokenized_final'].tolist()
description_list_final_13 = BELAIRPARKSIDE_df['tokenized_final'].tolist()
description_list_final_14 = BELLONAGITTINGS_df['tokenized_final'].tolist()
description_list_final_15 = BEREA_df['tokenized_final'].tolist()
description_list_final_16 = BETTERWAVERLY_df['tokenized_final'].tolist()
description_list_final_17 = BEVERLYHILLS_df['tokenized_final'].tolist()
description_list_final_18 = BIDDLESTREET_df['tokenized_final'].tolist()
description_list_final_19 = BLYTHEWOOD_df['tokenized_final'].tolist()
description_list_final_20 = BOLTONHILL_df['tokenized_final'].tolist()
description_list_final_21 = BOOTHBOYD_df['tokenized_final'].tolist()
description_list_final_22 = BREWERSHILL_df['tokenized_final'].tolist()
description_list_final_23 = BRIDGEVIEWGREENLAWN_df['tokenized_final'].tolist()
description_list_final_24 = BROADWAYEAST_df['tokenized_final'].tolist()
description_list_final_25 = BROENINGMANOR_df['tokenized_final'].tolist()
description_list_final_26 = BROOKLYN_df['tokenized_final'].tolist()
description_list_final_27 = BURLEITHLEIGHTON_df['tokenized_final'].tolist()
description_list_final_28 = CALLAWAYGARRISON_df['tokenized_final'].tolist()
description_list_final_29 = CAMERONVILLAGE_df['tokenized_final'].tolist()
description_list_final_30 = CANTON_df['tokenized_final'].tolist()
description_list_final_31 = CANTONINDUSTRIALAREA_df['tokenized_final'].tolist()
description_list_final_32 = CARROLLCAMDENINDUSTRIALAR_df['tokenized_final'].tolist()
description_list_final_33 = CARROLLPARK_df['tokenized_final'].tolist()
description_list_final_34 = CARROLLSOUTHHILTON_df['tokenized_final'].tolist()
description_list_final_35 = CARROLLTONRIDGE_df['tokenized_final'].tolist()
description_list_final_36 = CEDARCROFT_df['tokenized_final'].tolist()
description_list_final_37 = CEDMONT_df['tokenized_final'].tolist()
description_list_final_38 = CEDONIA_df['tokenized_final'].tolist()
description_list_final_39 = CENTRALFORESTPARK_df['tokenized_final'].tolist()
description_list_final_40 = CENTRALPARKHEIGHTS_df['tokenized_final'].tolist()
description_list_final_41 = CHARLESNORTH_df['tokenized_final'].tolist()
description_list_final_42 = CHARLESVILLAGE_df['tokenized_final'].tolist()
description_list_final_43 = CHERRYHILL_df['tokenized_final'].tolist()
description_list_final_44 = CHESWOLDE_df['tokenized_final'].tolist()
description_list_final_45 = CHINQUAPINPARKBELVEDERE_df['tokenized_final'].tolist()
description_list_final_46 = CHRISTOPHER_df['tokenized_final'].tolist()
description_list_final_47 = CLAREMONT_df['tokenized_final'].tolist()
description_list_final_48 = CLAREMONTFREEDOM_df['tokenized_final'].tolist()
description_list_final_49 = CLIFTONPARK_df['tokenized_final'].tolist()
description_list_final_50 = COLDSPRING_df['tokenized_final'].tolist()
description_list_final_51 = COLDSTREAMHOMESTEADMONTEBELL_df['tokenized_final'].tolist()
description_list_final_52 = CONCERNEDCITIZENSOFFORESTP_df['tokenized_final'].tolist()
description_list_final_53 = COPPINHEIGHTSASHCOEAST_df['tokenized_final'].tolist()
description_list_final_54 = CROSSCOUNTRY_df['tokenized_final'].tolist()
description_list_final_55 = CROSSKEYS_df['tokenized_final'].tolist()
description_list_final_56 = CURTISBAY_df['tokenized_final'].tolist()
description_list_final_57 = CURTISBAYINDUSTRIAL_df['tokenized_final'].tolist()
description_list_final_58 = CYLBURN_df['tokenized_final'].tolist()
description_list_final_59 = DARLEYPARK_df['tokenized_final'].tolist()
description_list_final_60 = DICKEYVILLE_df['tokenized_final'].tolist()
description_list_final_61 = DOLFIELD_df['tokenized_final'].tolist()
description_list_final_62 = DORCHESTER_df['tokenized_final'].tolist()
description_list_final_63 = DOWNTOWN_df['tokenized_final'].tolist()
description_list_final_64 = DRUIDHEIGHTS_df['tokenized_final'].tolist()
description_list_final_65 = DRUIDHILLPARK_df['tokenized_final'].tolist()
description_list_final_66 = DUNBARBROADWAY_df['tokenized_final'].tolist()
description_list_final_67 = DUNDALKMARINETERMINAL_df['tokenized_final'].tolist()
description_list_final_68 = EASTARLINGTON_df['tokenized_final'].tolist()
description_list_final_69 = EASTBALTIMOREMIDWAY_df['tokenized_final'].tolist()
description_list_final_70 = EASTERWOOD_df['tokenized_final'].tolist()
description_list_final_71 = EASTWOOD_df['tokenized_final'].tolist()
description_list_final_72 = EDMONDSONVILLAGE_df['tokenized_final'].tolist()
description_list_final_73 = EDNORGARDENSLAKESIDE_df['tokenized_final'].tolist()
description_list_final_74 = ELLWOODPARKMONUMENT_df['tokenized_final'].tolist()
description_list_final_75 = EVERGREEN_df['tokenized_final'].tolist()
description_list_final_76 = EVERGREENLAWN_df['tokenized_final'].tolist()
description_list_final_77 = FAIRFIELDAREA_df['tokenized_final'].tolist()
description_list_final_78 = FAIRMONT_df['tokenized_final'].tolist()
description_list_final_79 = FALLSTAFF_df['tokenized_final'].tolist()
description_list_final_80 = FEDERALHILL_df['tokenized_final'].tolist()
description_list_final_81 = FEDERALHILLMONTGOMERY_df['tokenized_final'].tolist()
description_list_final_82 = FELLSPOINT_df['tokenized_final'].tolist()
description_list_final_83 = FORESTPARK_df['tokenized_final'].tolist()
description_list_final_84 = FORESTPARKGOLFCOURSE_df['tokenized_final'].tolist()
description_list_final_85 = FOURBYFOUR_df['tokenized_final'].tolist()
description_list_final_86 = FRANKFORD_df['tokenized_final'].tolist()
description_list_final_87 = FRANKLINSQUARE_df['tokenized_final'].tolist()
description_list_final_88 = FRANKLINTOWN_df['tokenized_final'].tolist()
description_list_final_89 = FRANKLINTOWNROAD_df['tokenized_final'].tolist()
description_list_final_90 = GARWYNOAKS_df['tokenized_final'].tolist()
description_list_final_91 = GAYSTREET_df['tokenized_final'].tolist()
description_list_final_92 = GLEN_df['tokenized_final'].tolist()
description_list_final_93 = GLENOAKS_df['tokenized_final'].tolist()
description_list_final_94 = GLENHAMBELFORD_df['tokenized_final'].tolist()
description_list_final_95 = GRACELANDPARK_df['tokenized_final'].tolist()
description_list_final_96 = GREEKTOWN_df['tokenized_final'].tolist()
description_list_final_97 = GREENMOUNTCEMETERY_df['tokenized_final'].tolist()
description_list_final_98 = GREENMOUNTWEST_df['tokenized_final'].tolist()
description_list_final_99 = GREENSPRING_df['tokenized_final'].tolist()
description_list_final_100 = GROVEPARK_df['tokenized_final'].tolist()
description_list_final_101 = GUILFORD_df['tokenized_final'].tolist()
description_list_final_102 = GWYNNSFALLS_df['tokenized_final'].tolist()
description_list_final_103 = GWYNNSFALLSLEAKINPARK_df['tokenized_final'].tolist()
description_list_final_104 = HAMPDEN_df['tokenized_final'].tolist()
description_list_final_105 = HANLONLONGWOOD_df['tokenized_final'].tolist()
description_list_final_106 = HARFORDECHODALEPERRINGPARKW_df['tokenized_final'].tolist()
description_list_final_107 = HARLEMPARK_df['tokenized_final'].tolist()
description_list_final_108 = HARWOOD_df['tokenized_final'].tolist()
description_list_final_109 = HAWKINSPOINT_df['tokenized_final'].tolist()
description_list_final_110 = HERITAGECROSSING_df['tokenized_final'].tolist()
description_list_final_111 = HIGHLANDTOWN_df['tokenized_final'].tolist()
description_list_final_112 = HILLEN_df['tokenized_final'].tolist()
description_list_final_113 = HOESHEIGHTS_df['tokenized_final'].tolist()
description_list_final_114 = HOLABIRDINDUSTRIALPARK_df['tokenized_final'].tolist()
description_list_final_115 = HOLLINSMARKET_df['tokenized_final'].tolist()
description_list_final_116 = HOMELAND_df['tokenized_final'].tolist()
description_list_final_117 = HOPKINSBAYVIEW_df['tokenized_final'].tolist()
description_list_final_118 = HOWARDPARK_df['tokenized_final'].tolist()
description_list_final_119 = HUNTINGRIDGE_df['tokenized_final'].tolist()
description_list_final_120 = IDLEWOOD_df['tokenized_final'].tolist()
description_list_final_121 = INNERHARBOR_df['tokenized_final'].tolist()
description_list_final_122 = IRVINGTON_df['tokenized_final'].tolist()
description_list_final_123 = JOHNSHOPKINSHOMEWOOD_df['tokenized_final'].tolist()
description_list_final_124 = JOHNSTONSQUARE_df['tokenized_final'].tolist()
description_list_final_125 = JONESFALLSAREA_df['tokenized_final'].tolist()
description_list_final_126 = JONESTOWN_df['tokenized_final'].tolist()
description_list_final_127 = JOSEPHLEE_df['tokenized_final'].tolist()
description_list_final_128 = KENILWORTHPARK_df['tokenized_final'].tolist()
description_list_final_129 = KERNEWOOD_df['tokenized_final'].tolist()
description_list_final_130 = KESWICK_df['tokenized_final'].tolist()
description_list_final_131 = KRESSON_df['tokenized_final'].tolist()
description_list_final_132 = LAFAYETTE_df['tokenized_final'].tolist()
description_list_final_133 = LAKEEVESHAM_df['tokenized_final'].tolist()
description_list_final_134 = LAKEWALKER_df['tokenized_final'].tolist()
description_list_final_135 = LAKELAND_df['tokenized_final'].tolist()
description_list_final_136 = LANGSTONHUGHES_df['tokenized_final'].tolist()
description_list_final_137 = LAURAVILLE_df['tokenized_final'].tolist()
description_list_final_138 = LEVINDALE_df['tokenized_final'].tolist()
description_list_final_139 = LIBERTYSQUARE_df['tokenized_final'].tolist()
description_list_final_140 = LITTLEITALY_df['tokenized_final'].tolist()
description_list_final_141 = LOCHRAVEN_df['tokenized_final'].tolist()
description_list_final_142 = LOCUSTPOINT_df['tokenized_final'].tolist()
description_list_final_143 = LOCUSTPOINTINDUSTRIALAREA_df['tokenized_final'].tolist()
description_list_final_144 = LOWERHERRINGRUNPARK_df['tokenized_final'].tolist()
description_list_final_145 = LOYOLANOTREDAME_df['tokenized_final'].tolist()
description_list_final_146 = LUCILLEPARK_df['tokenized_final'].tolist()
description_list_final_147 = MADISONPARK_df['tokenized_final'].tolist()
description_list_final_148 = MADISONEASTEND_df['tokenized_final'].tolist()
description_list_final_149 = MAYFIELD_df['tokenized_final'].tolist()
description_list_final_150 = MCELDERRYPARK_df['tokenized_final'].tolist()
description_list_final_151 = MEDFIELD_df['tokenized_final'].tolist()
description_list_final_152 = MEDFORD_df['tokenized_final'].tolist()
description_list_final_153 = MIDDLEBRANCHREEDBIRDPARKS_df['tokenized_final'].tolist()
description_list_final_154 = MIDDLEEAST_df['tokenized_final'].tolist()
description_list_final_155 = MIDGOVANS_df['tokenized_final'].tolist()
description_list_final_156 = MIDTOWNBELVEDERE_df['tokenized_final'].tolist()
description_list_final_157 = MIDTOWNEDMONDSON_df['tokenized_final'].tolist()
description_list_final_158 = MILLHILL_df['tokenized_final'].tolist()
description_list_final_159 = MILTONMONTFORD_df['tokenized_final'].tolist()
description_list_final_160 = MONDAWMIN_df['tokenized_final'].tolist()
description_list_final_161 = MONTEBELLO_df['tokenized_final'].tolist()
description_list_final_162 = MORAVIAWALTHER_df['tokenized_final'].tolist()
description_list_final_163 = MORGANPARK_df['tokenized_final'].tolist()
description_list_final_164 = MORGANSTATEUNIVERSITY_df['tokenized_final'].tolist()
description_list_final_165 = MORRELLPARK_df['tokenized_final'].tolist()
description_list_final_166 = MOSHER_df['tokenized_final'].tolist()
description_list_final_167 = MOUNTHOLLY_df['tokenized_final'].tolist()
description_list_final_168 = MOUNTVERNON_df['tokenized_final'].tolist()
description_list_final_169 = MOUNTWASHINGTON_df['tokenized_final'].tolist()
description_list_final_170 = MOUNTWINANS_df['tokenized_final'].tolist()
description_list_final_171 = MTPLEASANTPARK_df['tokenized_final'].tolist()
description_list_final_172 = NEWNORTHWOOD_df['tokenized_final'].tolist()
description_list_final_173 = NEWSOUTHWESTMOUNTCLARE_df['tokenized_final'].tolist()
description_list_final_174 = NORTHHARFORDROAD_df['tokenized_final'].tolist()
description_list_final_175 = NORTHROLANDPARKPOPLARHILL_df['tokenized_final'].tolist()
description_list_final_176 = NORTHWESTCOMMUNITYACTION_df['tokenized_final'].tolist()
description_list_final_177 = OAKENSHAWE_df['tokenized_final'].tolist()
description_list_final_178 = OAKLEE_df['tokenized_final'].tolist()
description_list_final_179 = OLDTOWN_df['tokenized_final'].tolist()
description_list_final_180 = OLIVER_df['tokenized_final'].tolist()
description_list_final_181 = ORANGEVILLE_df['tokenized_final'].tolist()
description_list_final_182 = ORANGEVILLEINDUSTRIALAREA_df['tokenized_final'].tolist()
description_list_final_183 = ORIGINALNORTHWOOD_df['tokenized_final'].tolist()
description_list_final_184 = OTTERBEIN_df['tokenized_final'].tolist()
description_list_final_185 = OVERLEA_df['tokenized_final'].tolist()
description_list_final_186 = PANWAYBRADDISHAVENUE_df['tokenized_final'].tolist()
description_list_final_187 = PARKCIRCLE_df['tokenized_final'].tolist()
description_list_final_188 = PARKLANE_df['tokenized_final'].tolist()
description_list_final_189 = PARKSIDE_df['tokenized_final'].tolist()
description_list_final_190 = PARKVIEWWOODBROOK_df['tokenized_final'].tolist()
description_list_final_191 = PATTERSONPARK_df['tokenized_final'].tolist()
description_list_final_192 = PATTERSONPLACE_df['tokenized_final'].tolist()
description_list_final_193 = PENLUCY_df['tokenized_final'].tolist()
description_list_final_194 = PENNNORTH_df['tokenized_final'].tolist()
description_list_final_195 = PENNFALLSWAY_df['tokenized_final'].tolist()
description_list_final_196 = PENROSE_df['tokenized_final'].tolist()
description_list_final_197 = PENROSEFAYETTESTREETOUTREAC_df['tokenized_final'].tolist()
description_list_final_198 = PERKINSHOMES_df['tokenized_final'].tolist()
description_list_final_199 = PERRINGLOCH_df['tokenized_final'].tolist()
description_list_final_200 = PIMLICOGOODNEIGHBORS_df['tokenized_final'].tolist()
description_list_final_201 = PLEASANTVIEWGARDENS_df['tokenized_final'].tolist()
description_list_final_202 = POPPLETON_df['tokenized_final'].tolist()
description_list_final_203 = PORTCOVINGTON_df['tokenized_final'].tolist()
description_list_final_204 = PULASKIINDUSTRIALAREA_df['tokenized_final'].tolist()
description_list_final_205 = PURNELL_df['tokenized_final'].tolist()
description_list_final_206 = RADNORWINSTON_df['tokenized_final'].tolist()
description_list_final_207 = RAMBLEWOOD_df['tokenized_final'].tolist()
description_list_final_208 = REISTERSTOWNSTATION_df['tokenized_final'].tolist()
description_list_final_209 = REMINGTON_df['tokenized_final'].tolist()
description_list_final_210 = RESERVOIRHILL_df['tokenized_final'].tolist()
description_list_final_211 = RICHNORSPRINGS_df['tokenized_final'].tolist()
description_list_final_212 = RIVERSIDE_df['tokenized_final'].tolist()
description_list_final_213 = ROGNELHEIGHTS_df['tokenized_final'].tolist()
description_list_final_214 = ROLANDPARK_df['tokenized_final'].tolist()
description_list_final_215 = ROLANDPARKPOPLARHILL_df['tokenized_final'].tolist()
description_list_final_216 = ROSEBANK_df['tokenized_final'].tolist()
description_list_final_217 = ROSEMONT_df['tokenized_final'].tolist()
description_list_final_218 = ROSEMONTEAST_df['tokenized_final'].tolist()
description_list_final_219 = ROSEMONTHOMEOWNERSTENANTS_df['tokenized_final'].tolist()
description_list_final_220 = SABINAMATTFELDT_df['tokenized_final'].tolist()
description_list_final_221 = SAINTAGNES_df['tokenized_final'].tolist()
description_list_final_222 = SAINTHELENA_df['tokenized_final'].tolist()
description_list_final_223 = SAINTJOSEPHS_df['tokenized_final'].tolist()
description_list_final_224 = SAINTPAUL_df['tokenized_final'].tolist()
description_list_final_225 = SANDTOWNWINCHESTER_df['tokenized_final'].tolist()
description_list_final_226 = SBIC_df['tokenized_final'].tolist()
description_list_final_227 = SETONBUSINESSPARK_df['tokenized_final'].tolist()
description_list_final_228 = SETONHILL_df['tokenized_final'].tolist()
description_list_final_229 = SHARPLEADENHALL_df['tokenized_final'].tolist()
description_list_final_230 = SHIPLEYHILL_df['tokenized_final'].tolist()
description_list_final_231 = SOUTHCLIFTONPARK_df['tokenized_final'].tolist()
description_list_final_232 = SPRINGGARDENINDUSTRIALAREA_df['tokenized_final'].tolist()
description_list_final_233 = STADIUMAREA_df['tokenized_final'].tolist()
description_list_final_234 = STONEWOODPENTWOODWINSTON_df['tokenized_final'].tolist()
description_list_final_235 = TAYLORHEIGHTS_df['tokenized_final'].tolist()
description_list_final_236 = TENHILLS_df['tokenized_final'].tolist()
description_list_final_237 = THEORCHARDS_df['tokenized_final'].tolist()
description_list_final_238 = TOWANDAGRANTLEY_df['tokenized_final'].tolist()
description_list_final_239 = TREMONT_df['tokenized_final'].tolist()
description_list_final_240 = TUSCANYCANTERBURY_df['tokenized_final'].tolist()
description_list_final_241 = UNIONSQUARE_df['tokenized_final'].tolist()
description_list_final_242 = UNIVERSITYOFMARYLAND_df['tokenized_final'].tolist()
description_list_final_243 = UPLANDS_df['tokenized_final'].tolist()
description_list_final_244 = UPPERFELLSPOINT_df['tokenized_final'].tolist()
description_list_final_245 = UPTON_df['tokenized_final'].tolist()
description_list_final_246 = VILLAGESOFHOMELAND_df['tokenized_final'].tolist()
description_list_final_247 = VIOLETVILLE_df['tokenized_final'].tolist()
description_list_final_248 = WAKEFIELD_df['tokenized_final'].tolist()
description_list_final_249 = WALBROOK_df['tokenized_final'].tolist()
description_list_final_250 = WALTHERSON_df['tokenized_final'].tolist()
description_list_final_251 = WASHINGTONHILL_df['tokenized_final'].tolist()
description_list_final_252 = WASHINGTONVILLAGE_df['tokenized_final'].tolist()
description_list_final_253 = WAVERLY_df['tokenized_final'].tolist()
description_list_final_254 = WESTARLINGTON_df['tokenized_final'].tolist()
description_list_final_255 = WESTFORESTPARK_df['tokenized_final'].tolist()
description_list_final_256 = WESTHILLS_df['tokenized_final'].tolist()
description_list_final_257 = WESTFIELD_df['tokenized_final'].tolist()
description_list_final_258 = WESTGATE_df['tokenized_final'].tolist()
description_list_final_259 = WESTPORT_df['tokenized_final'].tolist()
description_list_final_260 = WILSONHEIGHTS_df['tokenized_final'].tolist()
description_list_final_261 = WILSONPARK_df['tokenized_final'].tolist()
description_list_final_262 = WINCHESTER_df['tokenized_final'].tolist()
description_list_final_263 = WINDSORHILLS_df['tokenized_final'].tolist()
description_list_final_264 = WINSTONGOVANS_df['tokenized_final'].tolist()
description_list_final_265 = WOODBERRY_df['tokenized_final'].tolist()
description_list_final_266 = WOODBOURNEHEIGHTS_df['tokenized_final'].tolist()
description_list_final_267 = WOODBOURNEMCCABE_df['tokenized_final'].tolist()
description_list_final_268 = WOODMERE_df['tokenized_final'].tolist()
description_list_final_269 = WOODRING_df['tokenized_final'].tolist()
description_list_final_270 = WYMANPARK_df['tokenized_final'].tolist()
description_list_final_271 = WYNDHURST_df['tokenized_final'].tolist()
description_list_final_272 = YALEHEIGHTS_df['tokenized_final'].tolist()

In [64]:
# ABELL = ['ABELL']
word_list_final_1 = []
for sublist in description_list_final_1:
    for word in sublist:
        word_list_final_1.append(word)
word_frequency_dict_final_1 = FreqDist(word_list_final_1)
df_word_frequency_1 = pd.DataFrame(word_frequency_dict_final_1.items(),columns=['word','frequency'])
df_word_frequency_1 = df_word_frequency_1.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_1['index']
frequency_sum_1 = df_word_frequency_1['frequency'].sum()
df_word_frequency_1.head(5)


,word,frequency
0,replace,365
1,existing,277
2,inches,220
3,floor,213
4,gas,179


In [65]:
ALLENDALE = ['ALLENDALE']
word_list_final_2 = []
for sublist in description_list_final_2:
    for word in sublist:
        word_list_final_2.append(word)
word_frequency_dict_final_2 = FreqDist(word_list_final_2)
df_word_frequency_2 = pd.DataFrame(word_frequency_dict_final_2.items(),columns=['word','frequency'])
df_word_frequency_2 = df_word_frequency_2.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_2['index']
frequency_sum_2 = df_word_frequency_2['frequency'].sum()
df_word_frequency_2.head(5)



,word,frequency
0,replace,1006
1,btu,708
2,interior,674
3,original,646
4,water,516


In [66]:
ARCADIA = ['ARCADIA']
word_list_final_3 = []
for sublist in description_list_final_3:
    for word in sublist:
        word_list_final_3.append(word)
word_frequency_dict_final_3 = FreqDist(word_list_final_3)
df_word_frequency_3 = pd.DataFrame(word_frequency_dict_final_3.items(),columns=['word','frequency'])
df_word_frequency_3 = df_word_frequency_3.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_3['index']
frequency_sum_3 = df_word_frequency_3['frequency'].sum()
df_word_frequency_3.head(5)

,word,frequency
0,replace,309
1,btu,301
2,interior,259
3,existing,252
4,water,215


In [67]:
ARLINGTON = ['ARLINGTON']
word_list_final_4 = []
for sublist in description_list_final_4:
    for word in sublist:
        word_list_final_4.append(word)
word_frequency_dict_final_4 = FreqDist(word_list_final_4)
df_word_frequency_4 = pd.DataFrame(word_frequency_dict_final_4.items(),columns=['word','frequency'])
df_word_frequency_4 = df_word_frequency_4.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_4['index']
frequency_sum_4 = df_word_frequency_4['frequency'].sum()
df_word_frequency_4.head(5)


,word,frequency
0,replace,643
1,interior,383
2,btu,371
3,original,360
4,use,319


In [68]:
ARMISTEADGARDENS = ['ARMISTEAD GARDENS']
word_list_final_5 = []
for sublist in description_list_final_5:
    for word in sublist:
        word_list_final_5.append(word)
word_frequency_dict_final_5 = FreqDist(word_list_final_5)
df_word_frequency_5 = pd.DataFrame(word_frequency_dict_final_5.items(),columns=['word','frequency'])
df_word_frequency_5 = df_word_frequency_5.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_5['index']
frequency_sum_5 = df_word_frequency_5['frequency'].sum()
df_word_frequency_5.head(5)


,word,frequency
0,replace,797
1,amp,684
2,circuits,646
3,panel,594
4,'',558


In [69]:
ASHBURTON = ['ASHBURTON']
word_list_final_6 = []
for sublist in description_list_final_6:
    for word in sublist:
        word_list_final_6.append(word)
word_frequency_dict_final_6 = FreqDist(word_list_final_6)
df_word_frequency_6 = pd.DataFrame(word_frequency_dict_final_6.items(),columns=['word','frequency'])
df_word_frequency_6 = df_word_frequency_6.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_6['index']
frequency_sum_6 = df_word_frequency_6['frequency'].sum()
df_word_frequency_6.head(5)


,word,frequency
0,replace,629
1,btu,574
2,gas,427
3,water,392
4,service,352


In [70]:
BALTIMOREHIGHLANDS = ['BALTIMORE HIGHLANDS']
word_list_final_7 = []
for sublist in description_list_final_7:
    for word in sublist:
        word_list_final_7.append(word)
word_frequency_dict_final_7 = FreqDist(word_list_final_7)
df_word_frequency_7 = pd.DataFrame(word_frequency_dict_final_7.items(),columns=['word','frequency'])
df_word_frequency_7 = df_word_frequency_7.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_7['index']
frequency_sum_7 = df_word_frequency_7['frequency'].sum()
df_word_frequency_7.head(5)


,word,frequency
0,replace,816
1,interior,800
2,btu,728
3,original,634
4,floor,616


In [71]:
BALTIMORELINWOOD = ['BALTIMORE-LINWOOD']
word_list_final_8 = []
for sublist in description_list_final_8:
    for word in sublist:
        word_list_final_8.append(word)
word_frequency_dict_final_8 = FreqDist(word_list_final_8)
df_word_frequency_8 = pd.DataFrame(word_frequency_dict_final_8.items(),columns=['word','frequency'])
df_word_frequency_8 = df_word_frequency_8.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_8['index']
frequency_sum_8 = df_word_frequency_8['frequency'].sum()
df_word_frequency_8.head(5)


,word,frequency
0,interior,4339
1,inches,3486
2,gas,3329
3,btu,3248
4,water,2873


In [72]:
BARCLAY = ['BARCLAY']
word_list_final_9 = []
for sublist in description_list_final_9:
    for word in sublist:
        word_list_final_9.append(word)
word_frequency_dict_final_9 = FreqDist(word_list_final_9)
df_word_frequency_9 = pd.DataFrame(word_frequency_dict_final_9.items(),columns=['word','frequency'])
df_word_frequency_9 = df_word_frequency_9.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_9['index']
frequency_sum_9 = df_word_frequency_9['frequency'].sum()
df_word_frequency_9.head(5)


,word,frequency
0,replace,1818
1,interior,1582
2,original,1497
3,floor,1146
4,existing,1082
